### Importing necessary modules

In [2]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import gc
import random
import numpy as np 
import pandas as pd

In [3]:
if torch.cuda.is_available():
    # If CUDA is available, use a CUDA device
    device = torch.device("cuda")
else:
    # If CUDA is not available, use the CPU
    device = torch.device("cpu")

In [4]:
device

device(type='cpu')

### Preprocessing

In [ ]:
traindata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_train.csv',names= ['English','Hindi'],header = None)

In [ ]:
testdata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_test.csv',names = ['English','Hindi'],header = None)

In [ ]:
valdata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_valid.csv',names = ['English','Hindi'],header = None)

In [ ]:
traindata

In [1]:
def tokenize(word):
    tokens = []
    for x in word:
        tokens.append(x)
    return tokens

In [5]:
max_eng_len = 0
max_hin_len = 0
test_max_eng_len = 0
test_max_hin_len = 0
val_max_eng_len = 0
val_max_hin_len = 0

In [ ]:
for x in range(len(testdata)):
    temp = 0
    for y in testdata.iloc[x]['English']:
        temp+=1
    test_max_eng_len = max(test_max_eng_len,temp)
print(test_max_eng_len)

In [ ]:
for x in range(len(testdata)):
    temp = 0
    for y in testdata.iloc[x]['Hindi']:
        temp +=1
    test_max_hin_len = max(test_max_hin_len,temp)
print(test_max_hin_len)

In [ ]:
for x in range(len(valdata)):
    temp = 0
    for y in valdata.iloc[x]['English']:
        temp+=1
    val_max_eng_len = max(val_max_eng_len,temp)
print(val_max_eng_len)

In [ ]:
for x in range(len(valdata)):
    temp = 0
    for y in valdata.iloc[x]['Hindi']:
        temp+=1
    val_max_hin_len = max(val_max_hin_len,temp)
print(val_max_hin_len)

In [ ]:
English_vocab = []
for x in range(len(traindata)):
    temp = 0
    for y in traindata.iloc[x]['English']:
        temp += 1
        if y not in English_vocab:
            English_vocab.append(y)
    if(temp>max_eng_len):
        max_eng_len = max(max_eng_len,temp)
print(sorted(English_vocab))
print(max_eng_len)

In [ ]:
Hindi_vocab = []
for x in range(len(traindata)):
    temp = 0
    for y in traindata.iloc[x]['Hindi']:
        temp += 1
        if y not in Hindi_vocab:
            Hindi_vocab.append(y)
    max_hin_len = max(temp,max_hin_len)
for x in range(len(testdata)):
    for y in testdata.iloc[x]['Hindi']:
        if y not in Hindi_vocab:
            print(y)
            Hindi_vocab.append(y)

In [ ]:
English_vocab = sorted(English_vocab)
Hindi_vocab = sorted(Hindi_vocab)

In [ ]:
Eng_dict = {}
reverse_Eng = {}

for x in range(len(English_vocab)):
    Eng_dict[English_vocab[x]] = x+3
    reverse_Eng[x+3] = English_vocab[x]
Eng_dict['<sow>'] = 0
Eng_dict['<eow>'] = 1
Eng_dict['<pad>'] = 2
reverse_Eng[0] = '<sow>'
reverse_Eng[1] = '<eow>'
reverse_Eng[2] = '<pad>'

print(Eng_dict)
print(reverse_Eng)

In [ ]:
Hin_dict = {}
reverse_Hin = {}
for x in range(len(Hindi_vocab)):
    Hin_dict[Hindi_vocab[x]] = x+3
    reverse_Hin[x+3] = Hindi_vocab[x]
Hin_dict['<sow>'] = 0
Hin_dict['<eow>'] = 1
Hin_dict['<pad>'] = 2
reverse_Hin[0] = '<sow>'
reverse_Hin[1] = '<eow>'
reverse_Hin[2] = '<pad>'
print(Hin_dict)
print(reverse_Hin)

In [ ]:
def Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens

In [ ]:
def Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),max_hin_len+1):
        tokens.append(Hin_dict['<pad>'])
    return tokens

In [ ]:
eng_word = []
hin_word = []
for x in range(len(traindata)):
    eng_word.append(Eng_tokenize(traindata.iloc[x]['English']))
    hin_word.append(Hin_tokenize(traindata.iloc[x]['Hindi']))

In [ ]:
eng_word = torch.tensor(eng_word)
hin_word = torch.tensor(hin_word)

In [ ]:
max_hin_len += 1
test_max_hin_len += 1
val_max_hin_len += 1

In [ ]:
def test_Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),test_max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens
def test_Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),test_max_hin_len):
        tokens.append(Hin_dict['<pad>'])
    return tokens
def val_Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),val_max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens
def val_Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),val_max_hin_len):
        tokens.append(Hin_dict['<pad>'])
    return tokens
val_eng_word = []
val_hin_word = []
for x in range(len(valdata)):
    val_eng_word.append(val_Eng_tokenize(valdata.iloc[x]['English']))
    val_hin_word.append(val_Hin_tokenize(valdata.iloc[x]['Hindi']))
val_eng_word = torch.tensor(val_eng_word)
val_hin_word = torch.tensor(val_hin_word)
test_eng_word = []
test_hin_word = []
for x in range(len(testdata)):
    test_eng_word.append(test_Eng_tokenize(testdata.iloc[x]['English']))
    test_hin_word.append(test_Hin_tokenize(testdata.iloc[x]['Hindi']))
test_eng_word = torch.tensor(test_eng_word)
test_hin_word = torch.tensor(test_hin_word)

### Encoder and Decoder

In [6]:
class Encoder(nn.Module):
    def __init__(self,char_embed_size,hidden_size,no_of_layers,dropout,rnn):
        super(Encoder,self).__init__()
        self.layer = no_of_layers
        self.rnn = rnn
        self.embedding = nn.Embedding(len(Eng_dict),char_embed_size).to(device)
        self.embedding.weight.requires_grad = True
        self.drop = nn.Dropout(dropout)
        self.LSTM = nn.LSTM(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
        self.RNN = nn.RNN(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
        self.GRU = nn.GRU(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
    def forward(self,input,hidden,cell):
        embedded = self.embedding(input)
        embedded1 = self.drop(embedded)
        cell1 = cell
        if(self.rnn == 'RNN'):
            output,hidden1 = self.RNN(embedded1,hidden)
        elif(self.rnn == 'LSTM'):
            output,(hidden1,cell1) = self.LSTM(embedded1,(hidden,cell))
        elif(self.rnn == 'GRU'):
            output,hidden1 = self.GRU(embedded1,hidden)
        return output,(hidden1,cell1)

In [7]:
class Decoder(nn.Module):
    def __init__(self,char_embed_size,hidden_size,no_of_layers,dropout,batchsize,rnn):
        super(Decoder,self).__init__()
        self.layer = no_of_layers
        self.batchsize = batchsize
        self.hidden_size = hidden_size
        self.rnn = rnn
        self.embedding = nn.Embedding(len(Hin_dict),char_embed_size).to(device)
        self.drop = nn.Dropout(dropout)
        self.embedding.weight.requires_grad = True
        self.LSTM = nn.LSTM(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        self.RNN = nn.RNN(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        self.GRU = nn.GRU(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        #2*hidden_size
        self.linear = nn.Linear(hidden_size,len(Hin_dict),bias=True).to(device)
        # dim = 2 
        self.softmax = nn.Softmax(dim = 2).to(device)
    def forward(self,input,hidden,cell,OGhidden,matrix):
        embedded = self.embedding(input)
        s1 = OGhidden.size()[1]
        s2 = OGhidden.size()[2]
        embedded1 = torch.cat((embedded,OGhidden[0].resize(s1,1,s2),OGhidden[1].resize(s1,1,s2)),dim = 2)
        embedded2 = self.drop(embedded1)
        cell1 = cell
        if(self.rnn == 'LSTM'):
            output,(hidden1,cell1) = self.LSTM(embedded2,(hidden,cell))
        elif(self.rnn == 'RNN'):
            output,hidden1 = self.RNN(embedded2,hidden)
        elif(self.rnn == 'GRU'):
            output,hidden1 = self.GRU(embedded2,hidden)
        output1 = self.linear(output)
        return output1,(hidden1,cell1)
        output2 = self.softmax(output1)
        return output2,hidden11
        
    #changed GRU char_embed_size
    #changed forward embedded

In [11]:
def getword(characters):
    return "".join(characters)

In [8]:
def accuracy(target,predictions,flag):
    total = 0
    for x in range(len(target)):
        if(torch.equal(target[x],predictions[x])):
            total += 1
    return total

In [9]:
def translate(target,predictions,df):
    i = len(df)
    for x in range(len(predictions)):
        original = []
        for y in target[x]:
            if(y != 1):
                original.append(y)
            else:
                break
        predicted = []
        for y in predictions[x]:
            if(y != 1):
                predicted.append(y)
            else:
                break
        df.loc[i,['Original']] = getword([reverse_Hin[x.item()] for x in original])
        df.loc[i,['Predicted']] = getword([reverse_Hin[x.item()] for x in predicted])
        i+=1
    return df
    

In [10]:
def Seq2Seq(attention,test_eng_word,test_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers):
    with torch.no_grad():
        total_loss = 0
        total_acc = 0
        df = pd.DataFrame()
        en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
        en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
        for x in range(0,len(testdata),batchsize):
            loss = 0
            input_tensor = test_eng_word[x:x+batchsize].to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            del(input_tensor)
            output = torch.split(output,[hidden_size,hidden_size],dim = 2)
            output = torch.add(output[0],output[1])/2
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            cell1 = torch.add(cell[0],cell[1])/2
            OGhidden = hidden1
            predicted = []
            predictions = []
            if(attention == True):
                temp = output
            else:
                temp = OGhidden
            for i in range(test_max_hin_len):
                output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,temp,False)
                predicted.append(output1)
                output2 = decoder.softmax(output1)
                output3 = torch.argmax(output2,dim = 2)
                predictions.append(output3)
                input2 = output3
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(test_max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(test_hin_word[x:x+batchsize].to(device),predictions,x)
            df = translate(test_hin_word[x:x+batchsize],predictions,df)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,test_hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
        test_loss = total_loss/(len(testdata)*test_max_hin_len)
        test_accuracy = (total_acc/len(testdata))*100
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden1)
        del(cell1)
        del(OGhidden)
        del(output)
        del(cell)
        return test_loss,test_accuracy,df